In [30]:
# manipulacja danymi
import numpy as np
import pandas as pd

# wizualizacja
import matplotlib.pyplot as plt
import seaborn as sns

# podział danych na zbiory treningowe/walidacyjne/testowe
from sklearn.model_selection import train_test_split, GridSearchCV

# budowa Pipeline
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer

# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures, PowerTransformer

# redukcja wymiarowości
from sklearn.decomposition import PCA

# model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# ewaluacja
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, f1_score, roc_auc_score

It's gonna be PENGUINGS againn, google 'penguings Cumberbatch' and have fun:))

In [31]:
penguings = pd.read_csv(r'C:\_PRYWATNE\LANGUAGES\Python\_DATA_SCIENCE\py\MOD_15\penguins.csv')
penguings

,CulmenLength,CulmenDepth,FlipperLength,BodyMass,Species
0,39.1,18.7,181.0,3750.0,0
1,39.5,17.4,186.0,3800.0,0
2,40.3,18.0,195.0,3250.0,0
3,NaN,NaN,NaN,NaN,0
4,36.7,19.3,193.0,3450.0,0
...,...,...,...,...,...
339,55.8,19.8,207.0,4000.0,2
340,43.5,18.1,202.0,3400.0,2
341,49.6,18.2,193.0,3775.0,2
342,50.8,19.0,210.0,4100.0,2


what do `random state` and `stratify` actually do?

->here I'm dividing the raw dataset into all th eneeded parts

In [32]:
X = penguings.drop(['Species'], axis=1).copy()
y = penguings['Species'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)


In [33]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CulmenLength   342 non-null    float64
 1   CulmenDepth    342 non-null    float64
 2   FlipperLength  342 non-null    float64
 3   BodyMass       342 non-null    float64
dtypes: float64(4)
memory usage: 10.9 KB


In [34]:
y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 344 entries, 0 to 343
Series name: Species
Non-Null Count  Dtype
--------------  -----
344 non-null    int64
dtypes: int64(1)
memory usage: 2.8 KB


In [35]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 258 entries, 326 to 140
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CulmenLength   256 non-null    float64
 1   CulmenDepth    256 non-null    float64
 2   FlipperLength  256 non-null    float64
 3   BodyMass       256 non-null    float64
dtypes: float64(4)
memory usage: 10.1 KB


In [36]:
y_train.info()

<class 'pandas.core.series.Series'>
Index: 258 entries, 326 to 140
Series name: Species
Non-Null Count  Dtype
--------------  -----
258 non-null    int64
dtypes: int64(1)
memory usage: 4.0 KB


next, preparation of all needed elements to construct a pipeline.

1) numerical values preparation (in this case there's no cat values except for the results)
    a) fill missing values
    b) polynomoal features - wtf?
    c) scaler 1 - wtf?
    d) pca - wtf?
    e) scaler 2 - same as scaler 1, but after pca. What's going on?

2) no cat values, appears to be a bit less work for me:))

3) transformer //data_preparation / preprocessor// - creates the actual 'transformer fuction' which will be later applied to the `pipeline model`. This merges the numerical and categorical preparation workflows.

4) creation of the model

5) fit the model

6) check up the model:
    a) calculate metrics
    b)visualise the model or its' performance

In [37]:
num_features = ['CulmenLength', 'CulmenDepth', 'FlipperLength', 'BodyMass']

#prepare num values
num_preparation = Pipeline(steps=[
    ('fill_missing', SimpleImputer(strategy='mean')),
])

#no cat values to be prepared

##transformer
data_preparation = ColumnTransformer(transformers=[
    ('numeric_preprocessing', num_preparation, num_features)
])

model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=50)


model_pipeline = Pipeline(steps=[
    ('preprocessor', data_preparation),
    ('model', model)
])


In [38]:
list(model_pipeline.get_params().keys())

['memory',
 'steps',
 'verbose',
 'preprocessor',
 'model',
 'preprocessor__n_jobs',
 'preprocessor__remainder',
 'preprocessor__sparse_threshold',
 'preprocessor__transformer_weights',
 'preprocessor__transformers',
 'preprocessor__verbose',
 'preprocessor__verbose_feature_names_out',
 'preprocessor__numeric_preprocessing',
 'preprocessor__numeric_preprocessing__memory',
 'preprocessor__numeric_preprocessing__steps',
 'preprocessor__numeric_preprocessing__verbose',
 'preprocessor__numeric_preprocessing__fill_missing',
 'preprocessor__numeric_preprocessing__fill_missing__add_indicator',
 'preprocessor__numeric_preprocessing__fill_missing__copy',
 'preprocessor__numeric_preprocessing__fill_missing__fill_value',
 'preprocessor__numeric_preprocessing__fill_missing__keep_empty_features',
 'preprocessor__numeric_preprocessing__fill_missing__missing_values',
 'preprocessor__numeric_preprocessing__fill_missing__strategy',
 'preprocessor__numeric_preprocessing__fill_missing__verbose',
 'model_

In [39]:
params = {
    'preprocessor__numeric_preprocessing__fill_missing__strategy': ['mean', 'median'],
    'model__n_estimators': [25, 50, 75],
    'model__base_estimator__max_depth': [0.01, 0.1, 1, 10]
}

grid_search = GridSearchCV(model_pipeline, params, cv=10, n_jobs=1, verbose=10, scoring='f1_macro')
grid_search.fit(X_train, y_train)
print('wybrane parametry', grid_search.best_params_)
model = grid_search.best_estimator_

Fitting 10 folds for each of 24 candidates, totalling 240 fits
[CV 1/10; 1/24] START model__base_estimator__max_depth=0.01, model__n_estimators=25, preprocessor__numeric_preprocessing__fill_missing__strategy=mean
[CV 1/10; 1/24] END model__base_estimator__max_depth=0.01, model__n_estimators=25, preprocessor__numeric_preprocessing__fill_missing__strategy=mean;, score=nan total time=   0.0s
[CV 2/10; 1/24] START model__base_estimator__max_depth=0.01, model__n_estimators=25, preprocessor__numeric_preprocessing__fill_missing__strategy=mean
[CV 2/10; 1/24] END model__base_estimator__max_depth=0.01, model__n_estimators=25, preprocessor__numeric_preprocessing__fill_missing__strategy=mean;, score=nan total time=   0.0s
[CV 3/10; 1/24] START model__base_estimator__max_depth=0.01, model__n_estimators=25, preprocessor__numeric_preprocessing__fill_missing__strategy=mean
[CV 3/10; 1/24] END model__base_estimator__max_depth=0.01, model__n_estimators=25, preprocessor__numeric_preprocessing__fill_miss

c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\krzys\anaconda3\Lib\site-p

[CV 8/10; 3/24] END model__base_estimator__max_depth=0.01, model__n_estimators=50, preprocessor__numeric_preprocessing__fill_missing__strategy=mean;, score=nan total time=   0.0s
[CV 9/10; 3/24] START model__base_estimator__max_depth=0.01, model__n_estimators=50, preprocessor__numeric_preprocessing__fill_missing__strategy=mean
[CV 9/10; 3/24] END model__base_estimator__max_depth=0.01, model__n_estimators=50, preprocessor__numeric_preprocessing__fill_missing__strategy=mean;, score=nan total time=   0.0s
[CV 10/10; 3/24] START model__base_estimator__max_depth=0.01, model__n_estimators=50, preprocessor__numeric_preprocessing__fill_missing__strategy=mean
[CV 10/10; 3/24] END model__base_estimator__max_depth=0.01, model__n_estimators=50, preprocessor__numeric_preprocessing__fill_missing__strategy=mean;, score=nan total time=   0.0s
[CV 1/10; 4/24] START model__base_estimator__max_depth=0.01, model__n_estimators=50, preprocessor__numeric_preprocessing__fill_missing__strategy=median
[CV 1/10;

c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\krzys\anaconda3\Lib\site-p

[CV 1/10; 6/24] END model__base_estimator__max_depth=0.01, model__n_estimators=75, preprocessor__numeric_preprocessing__fill_missing__strategy=median;, score=nan total time=   0.0s
[CV 2/10; 6/24] START model__base_estimator__max_depth=0.01, model__n_estimators=75, preprocessor__numeric_preprocessing__fill_missing__strategy=median
[CV 2/10; 6/24] END model__base_estimator__max_depth=0.01, model__n_estimators=75, preprocessor__numeric_preprocessing__fill_missing__strategy=median;, score=nan total time=   0.0s
[CV 3/10; 6/24] START model__base_estimator__max_depth=0.01, model__n_estimators=75, preprocessor__numeric_preprocessing__fill_missing__strategy=median
[CV 3/10; 6/24] END model__base_estimator__max_depth=0.01, model__n_estimators=75, preprocessor__numeric_preprocessing__fill_missing__strategy=median;, score=nan total time=   0.0s
[CV 4/10; 6/24] START model__base_estimator__max_depth=0.01, model__n_estimators=75, preprocessor__numeric_preprocessing__fill_missing__strategy=median
[

...

c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


[CV 7/10; 24/24] END model__base_estimator__max_depth=10, model__n_estimators=75, preprocessor__numeric_preprocessing__fill_missing__strategy=median;, score=0.914 total time=   0.1s
[CV 8/10; 24/24] START model__base_estimator__max_depth=10, model__n_estimators=75, preprocessor__numeric_preprocessing__fill_missing__strategy=median
[CV 8/10; 24/24] END model__base_estimator__max_depth=10, model__n_estimators=75, preprocessor__numeric_preprocessing__fill_missing__strategy=median;, score=1.000 total time=   0.0s
[CV 9/10; 24/24] START model__base_estimator__max_depth=10, model__n_estimators=75, preprocessor__numeric_preprocessing__fill_missing__strategy=median


c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


[CV 9/10; 24/24] END model__base_estimator__max_depth=10, model__n_estimators=75, preprocessor__numeric_preprocessing__fill_missing__strategy=median;, score=1.000 total time=   0.0s
[CV 10/10; 24/24] START model__base_estimator__max_depth=10, model__n_estimators=75, preprocessor__numeric_preprocessing__fill_missing__strategy=median
[CV 10/10; 24/24] END model__base_estimator__max_depth=10, model__n_estimators=75, preprocessor__numeric_preprocessing__fill_missing__strategy=median;, score=1.000 total time=   0.0s
wybrane parametry {'model__base_estimator__max_depth': 10, 'model__n_estimators': 25, 'preprocessor__numeric_preprocessing__fill_missing__strategy': 'mean'}


c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
120 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\krzys\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 405, in fit
    self._final_estimator

In [40]:
X_train.info()
y_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 258 entries, 326 to 140
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CulmenLength   256 non-null    float64
 1   CulmenDepth    256 non-null    float64
 2   FlipperLength  256 non-null    float64
 3   BodyMass       256 non-null    float64
dtypes: float64(4)
memory usage: 10.1 KB
<class 'pandas.core.series.Series'>
Index: 258 entries, 326 to 140
Series name: Species
Non-Null Count  Dtype
--------------  -----
258 non-null    int64
dtypes: int64(1)
memory usage: 4.0 KB


In [41]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric_preprocessing',
                                                  Pipeline(steps=[('fill_missing',
                                                                   SimpleImputer())]),
                                                  ['CulmenLength',
                                                   'CulmenDepth',
                                                   'FlipperLength',
                                                   'BodyMass'])])),
                ('model',
                 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=10),
                                    n_estimators=25))])

In [42]:
model.get_params()

{'memory': None,
 'steps': [('preprocessor',
   ColumnTransformer(transformers=[('numeric_preprocessing',
                                    Pipeline(steps=[('fill_missing',
                                                     SimpleImputer())]),
                                    ['CulmenLength', 'CulmenDepth',
                                     'FlipperLength', 'BodyMass'])])),
  ('model',
   AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=10),
                      n_estimators=25))],
 'verbose': False,
 'preprocessor': ColumnTransformer(transformers=[('numeric_preprocessing',
                                  Pipeline(steps=[('fill_missing',
                                                   SimpleImputer())]),
                                  ['CulmenLength', 'CulmenDepth',
                                   'FlipperLength', 'BodyMass'])]),
 'model': AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=10),
                    n_estimators=25)